In [51]:
import requests
from bs4 import BeautifulSoup
import json

url = 'https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?id=15174;type=tournament'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [52]:
# Parser Code
links = []
player_links=[]
table_rows = soup.select('table.engineTable > tbody > tr.data1')
for row in table_rows:
    match_url = "https://www.espncricinfo.com" + row.select('td')[6].select('a')[0]['href']
    links.append(match_url)


In [53]:
for url in links:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    match = soup.find(
        'span', text='MATCH FLOW').parent.parent.next_sibling.find_all_next('div')
    team1 = match[0].select('span')[0].text.replace("innings", "")
    team2 = match[1].select('span')[0].text.replace("innings", "")
    tables = soup.select('div > table.ci-scorecard-table')
    first_inning_rows = [row for row in tables[0].select(
        'tbody > tr') if len(row.select('td')) >= 8]
    second_inning_rows = [row for row in tables[1].select(
        'tbody > tr') if len(row.select('td')) >= 8]

    try:

        for i in range(0, len(first_inning_rows)):
            row = first_inning_rows[i]
            cols = row.select('td')
            name = cols[0].select_one("a > span > span").text.replace('\xa0', '')
            link = "https://www.espncricinfo.com" + cols[0].select_one("a")["href"]
            player_links.append({"name": name, "team": team1, "link": link})

    except Exception as e:
        print(e)
  
    try:

        for i in range(0, len(second_inning_rows)):
            # print(len(first_inning_rows))
            row = second_inning_rows[i]
            cols = row.select('td')
            name = cols[0].select_one("a > span > span").text.replace('\xa0', '')
            link = "https://www.espncricinfo.com" + cols[0].select_one("a")["href"]
            player_links.append({"name": name, "team": team2, "link": link})

    except Exception as e:
        print(e)
    team1 = match[1].select('span')[0].text.replace("innings","")
    team2 = match[0].select('span')[0].text.replace("innings","")
    tables = soup.select('div > table.ds-table')
    first_inning_rows = [row for row in tables[1].select('tbody > tr') if len(row.select('td')) >= 11]
    second_inning_rows = [row for row in tables[3].select('tbody > tr') if len(row.select('td')) >=11]
    try:

        for i in range(0, len(first_inning_rows)):
            # print(len(first_inning_rows))
            row = first_inning_rows[i]
            cols = row.select('td')
            name = cols[0].select("a > span")[0].text.replace('\xa0', '')
            link = "https://www.espncricinfo.com" + cols[0].select("a")[0]["href"]
            player_links.append({"name": name, "team": team1, "link": link})

    except Exception as e:
        print(e)
    try:

        for i in range(0, len(second_inning_rows)):
            # print(len(first_inning_rows))
            row = second_inning_rows[i]
            cols = row.select('td')
            name = cols[0].select("a > span")[0].text.replace('\xa0', '')
            link = "https://www.espncricinfo.com" + cols[0].select("a")[0]["href"]
            player_links.append({"name": name, "team": team2, "link": link})

    except Exception as e:
        print(e)

In [54]:
seen_names = set()
unique_player_links = []

for player in player_links:
    name = player['name']
    if name not in seen_names:
        seen_names.add(name)
        unique_player_links.append(player)

In [71]:
try:
    for player in unique_player_links:
        link = player['link']
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
#         battingStyle = soup.find('div', class_='ds-grid').find(lambda tag: tag.name == 'div' and 'Batting Style' in tag.text).parent.select('span')[3].text
#         playingRole = soup.find('div', class_='ds-grid').find(lambda tag: tag.name == 'div' and 'Batting Style' in tag.text).parent.select('span')[5].text
        battingStyle_elem = soup.find('div', class_='ds-grid').find(lambda tag: tag.name == 'div' and 'Batting Style' in tag.text)
        if battingStyle_elem:
            battingStyle = battingStyle_elem.find('span').text
        else:
            battingStyle = ""
        bowlingStyle_WK_elem = soup.find('div', class_='ds-grid').find(lambda tag: tag.name == 'div' and any(x in tag.text for x in ['Fielding Position', 'Bowling Style']))
        if bowlingStyle_WK_elem:
            bowlingStyle_WK = bowlingStyle_WK_elem.find('span').text
        else:
            bowlingStyle_WK = ""
        playingRole_elem = soup.find('div', class_='ds-grid').find(lambda tag: tag.name == 'div' and 'Playing Role' in tag.text)
        if playingRole_elem:
            playingRole = playingRole_elem.find('span').text
        else:
            playingRole = ""
        
        player['battingStyle'] = battingStyle
        player['bowlingStyle/WK'] = bowlingStyle_WK
        player['playingRole'] = playingRole
       
        
except Exception as e:
    print(e)
# print(unique_player_links)

<img alt="Yastika Bhatia" class="" src="https://wassets.hscicdn.com/static/images/lazyimage-transparent.png" style="position:absolute;top:0;left:0;width:100%;height:100%"/>
<img alt="Hayley Matthews" class="" src="https://wassets.hscicdn.com/static/images/lazyimage-transparent.png" style="position:absolute;top:0;left:0;width:100%;height:100%"/>
<img alt="Nat Sciver-Brunt" class="" src="https://wassets.hscicdn.com/static/images/lazyimage-transparent.png" style="position:absolute;top:0;left:0;width:100%;height:100%"/>
<img alt="Harmanpreet Kaur" class="" src="https://wassets.hscicdn.com/static/images/lazyimage-transparent.png" style="position:absolute;top:0;left:0;width:100%;height:100%"/>
<img alt="Amelia Kerr" class="" src="https://wassets.hscicdn.com/static/images/lazyimage-transparent.png" style="position:absolute;top:0;left:0;width:100%;height:100%"/>


KeyboardInterrupt: 

In [56]:
with open('player_info.json', 'w') as f:
    json.dump(unique_player_links, f, indent=4)

C:\Users\Asus\AppData\Local\Temp\ipykernel_16116\691909413.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)


'WebDriver' object has no attribute 'find_element_by_css_selector'
